# Bronze Layer Ingestion

## 1 - Start SparkSession

In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .appName("olist-bronze-setup")
        .getOrCreate()
)

[16:50:08] [Thread-3] WARN  org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 2 - Create Namespace

In [2]:
spark.sql("SHOW CATALOGS").show()

+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [3]:
spark.sql("USE polaris")

[16:51:11] [Thread-3] INFO  org.apache.iceberg.rest.auth.AuthManagers - Loading AuthManager implementation: org.apache.iceberg.rest.auth.OAuth2Manager
[16:51:12] [Thread-3] INFO  org.apache.iceberg.CatalogUtil - Loading custom FileIO implementation: org.apache.iceberg.aws.s3.S3FileIO


DataFrame[]

In [4]:
spark.sql("""CREATE NAMESPACE IF NOT EXISTS bronze""")

DataFrame[]

In [5]:
spark.sql("SHOW NAMESPACES").show()

+---------+
|namespace|
+---------+
|   bronze|
+---------+



In [6]:
kafka_raw_df = (
    spark.read.format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "olist.oltp.customers")
    .option("startingOffsets", "earliest")
    .load()
)

In [7]:
kafka_raw_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [9]:
from pyspark.sql.avro.functions import from_avro
from pyspark.sql.functions import col

apicurio_opts = {
    "schema.registry.url": "http://apicurio:8081/apis/registry/v2",
    "value.subject.name.strategy": "io.apicurio.registry.serde.strategy.TopicIdStrategy"
}

decoded_raw = kafka_raw_df.select(
    from_avro(col("value"), "avro", apicurio_opts).alias("data")
)

decoded_raw.printSchema()

Py4JJavaError: An error occurred while calling o71.select.
: org.apache.avro.SchemaParseException: com.fasterxml.jackson.core.JsonParseException: Unrecognized token 'avro': was expecting (JSON String, Number, Array, Object or token 'null', 'true' or 'false')
 at [Source: REDACTED (`StreamReadFeature.INCLUDE_SOURCE_IN_LOCATION` disabled); line: 1, column: 5]
	at org.apache.avro.Schema$Parser.parse(Schema.java:1560)
	at org.apache.avro.Schema$Parser.parse(Schema.java:1515)
	at org.apache.spark.sql.avro.AvroDataToCatalyst.actualSchema$lzycompute(AvroDataToCatalyst.scala:61)
	at org.apache.spark.sql.avro.AvroDataToCatalyst.actualSchema(AvroDataToCatalyst.scala:60)
	at org.apache.spark.sql.avro.AvroDataToCatalyst.$anonfun$expectedSchema$1(AvroDataToCatalyst.scala:63)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.sql.avro.AvroDataToCatalyst.expectedSchema$lzycompute(AvroDataToCatalyst.scala:63)
	at org.apache.spark.sql.avro.AvroDataToCatalyst.expectedSchema(AvroDataToCata